In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

In [3]:
def vega(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

In [4]:
def bisection(S, K, T, r, C_market):
    left = 0.001
    right = 1_000

    for _ in range(15):
        sigma = 0.5 * (left + right)
        c_new = black_scholes_call(S, K, T, r, sigma)
        if c_new < C_market:
            left = sigma
        if c_new > C_market:
            right = sigma
    
    return sigma

In [5]:
def newton(S, K, T, r, C_market, initial_guess, tol=1e-6, max_iters=10):
    sigma = initial_guess
    for i in range(max_iters):
        c_new = black_scholes_call(S, K, T, r, sigma)
        v = vega(S, K, T, r, sigma)
        diff = c_new - C_market
        if abs(diff) < tol:
            return sigma, i+1
        if v < 1e-8:
            return sigma, i+1
        sigma -= diff / v
    return -1, max_iters

In [6]:
S = 105
K = 100
T = 0.25
r = 0.05
C_market = 10

In [7]:
initial_guess = bisection(S, K, T, r, C_market)
initial_guess

0.3366930236816406

In [8]:
implied_volatility, iters = newton(S, K, T, r, C_market, initial_guess)
implied_volatility, iters

(0.3153784095894166, 3)